In [1]:
sample_file = r"0000000d.ps3t"  # small file
large_sample_file = r"dxhr\bigfile.000"  # obscenely large file

In [2]:
class ByteViewer:
    config: dict = {
                "bytes_per_sep": 4,
                "bytes_per_row": 16,

                "startrow": 0x0,  # not completely tested
                "nrows": 0xF,

                "uppercase": True,

                "swap": False,
                "format": "L",  # relevant if swap is true. see documentation on struct for the other fmt codes

                "show_offset_row": False,  # not implemented
                "show_offset_column": True,
                "show_decoded_column": True,  # not implemented
                "encoding": "latin-1",  # not implemented 
            }
    colors: dict = {
        # must be CSS-safe color strings
        "zeros": "rgb(125,125,125)",
        "offset": "rgb(125,125,125)",
        "decoded": "rgb(125,125,125)",
    }

    def __init__(self, data: bytes):
        self.data: bytes = data

    @classmethod
    def from_path(cls, path):
        with open(path, 'rb') as f:
            start = cls.config["startrow"] * cls.config["bytes_per_row"]
            length = cls.config["nrows"] * cls.config["bytes_per_row"]
            f.seek(start)
            return cls(f.read(length))

    def _swap_data(self, data: bytes) -> bytes:
        from array import array
        import struct
        t = array(self.config["format"], data)
        t.byteswap()
        return b"".join([struct.pack(self.config["format"], i) for i in t])

    def _repr_html_(self):
        out_html = '<p style="font-family:monospace;">'

        d = self._swap_data(self.data) if self.config["swap"] else self.data

        # separate the bytes per chunk
        byte_sep_text = d.hex(" ", self.config["bytes_per_sep"])

        # turn uppercase if that's in the config
        byte_sep_text = byte_sep_text.upper() if self.config["uppercase"] else byte_sep_text

        chunks_per_row = self.config["bytes_per_row"]//self.config["bytes_per_sep"]

        sep_text_as_list = byte_sep_text.split()

        text_per_row = []
        for k in range(0, len(byte_sep_text.split()), chunks_per_row):
            text_per_row.append(" ".join(sep_text_as_list[k:k+chunks_per_row]))

        for row_num, row in enumerate(text_per_row):
            rnum = row_num + self.config["startrow"]
            out_html += f"<span style='color: {self.colors['offset']}'>{hex(rnum)[2:].zfill(8)}</span>: {row} <br />"

            if row_num == self.config['nrows']:
                break

        out_html += "</p>"
        return out_html

In [3]:
with open(sample_file, 'rb') as f:
    bv = ByteViewer(f.read())

bv

In [4]:
ByteViewer.from_path(large_sample_file)